# 第5章: 係り受け解析

日本語Wikipediaの「人工知能」に関する記事からテキスト部分を抜き出したファイルがai.ja.zipに収録されている． この文章をCaboChaやKNP等のツールを利用して係り受け解析を行い，その結果をai.ja.txt.parsedというファイルに保存せよ．このファイルを読み込み，以下の問に対応するプログラムを実装せよ．

KNP使うときのコマンド

`juman < ai.ja.txt | knp -simple -anaphora > ai.ja.txt.parsed`

In [25]:
import os
from typing import List, Dict

In [26]:
input_path = os.path.join(os.getcwd(), '../data/ai.ja.txt.parsed')

In [29]:
# 読み込み
def get_raw(path: str) -> str:
#     with open(path, mode='r', encoding='utf-8') as f:
    with open(path, mode='r') as f:
        return f.read()

In [39]:
# get_raw(input_path)

## 40. 係り受け解析結果の読み込み（形態素）
形態素を表すクラスMorphを実装せよ．このクラスは表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をメンバ変数に持つこととする．さらに，係り受け解析の結果（ai.ja.txt.parsed）を読み込み，各文をMorphオブジェクトのリストとして表現し，冒頭の説明文の形態素列を表示せよ．

In [86]:
class Morph:
    def __init__(self, morph_dict: dict):
        self.surface = morph_dict['surface'] # 表層形
        self.base = morph_dict['base'] # 基本形
        self.pos = morph_dict['pos'] # 品詞
        self.pos1 = morph_dict['pos1'] # 品詞細分類1

Morph({'surface': 'a', 'base': 'b', 'pos': 'c', 'pos1': 'd'})

In [40]:
raw = get_raw(input_path)

In [140]:
# 文ごとに分割
def get_sentenses(text: str) -> List[str]:
    return text.split('EOS\n')

get_sentenses('aEOS\nb') == ['a', 'b']

True

In [141]:
sentenses = get_sentenses(raw)
sentenses

['# S-ID:1 KNP:4.11-CF1.1 DATE:2020/12/18 SCORE:-1.74752\n* -1D <体言><用言:判>\n+ 1D <係:文節内><体言><EID:0>\n人工 じんこう 人工 名詞 6 普通名詞 1 * 0 * 0 "代表表記:人工/じんこう カテゴリ:抽象物" \n+ -1D <体言><用言:判><EID:1>\n知能 ちのう 知能 名詞 6 普通名詞 1 * 0 * 0 "代表表記:知能/ちのう カテゴリ:抽象物" \n',
 '# S-ID:2 KNP:4.11-CF1.1 DATE:2020/12/18 SCORE:-1.74752 ERROR:Cannot make mrph\n',
 '# S-ID:3 KNP:4.11-CF1.1 DATE:2020/12/18 SCORE:-306.79845\n* 3A <体言><係:同格連体>\n+ 1D <係:文節内><体言><EID:2>\n人工 じんこう 人工 名詞 6 普通名詞 1 * 0 * 0 "代表表記:人工/じんこう カテゴリ:抽象物" \n+ 5A <体言><係:同格連体><EID:1>\n知能 ちのう 知能 名詞 6 普通名詞 1 * 0 * 0 "代表表記:知能/ちのう カテゴリ:抽象物" \n* 3P <体言><係:連体>\n+ 3D <係:文節内><体言><EID:3>\n（ （ （ 特殊 1 括弧始 3 * 0 * 0 NIL \nじんこう じんこう じんこう 名詞 6 普通名詞 1 * 0 * 0 "代表表記:人口/じんこう カテゴリ:数量" \n+ 5P <体言><係:連体><EID:4>\nちのう ちのう ちのう 名詞 6 普通名詞 1 * 0 * 0 "代表表記:知能/ちのう カテゴリ:抽象物" \n、 、 、 特殊 1 読点 2 * 0 * 0 NIL \n、 、 、 特殊 1 読点 2 * 0 * 0 NIL \n* 3A <体言><係:同格連体>\n+ 5A <体言><係:同格連体><EID:5>\nAI AI AI 名詞 6 普通名詞 1 * 0 * 0 "疑似代表表記 代表表記:AI/AI 品詞変更:AI-AI-AI-15-1-0-0" \n* 15D <体言><係:ト格>\n+ 23D <体言><係:ト格><EID:6

In [180]:
# 文から、形態素の辞書リストを作る
def get_morph_dict_list(text: str) -> List[dict]:
    morph_list = text.split('\n') # 改行で分割
    ret_list = []
    for m in morph_list:
        if len(m) == 0:
            continue
        # #, *, +で始まる行は形態素でない情報なのでパス
        if m[:1] == '#' or m[:1] == '*' or m[:1] == '+':
            continue
        morp = m.split(' ')
        surface = morp[0]
        base = morp[2]
        pos = morp[3]
        pos1 = morp[5]
        ret_list.append({
            'surface': surface,
            'base': base,
            'pos': pos,
            'pos1': pos1
        })
    return ret_list

tes = '# S-ID:1 KNP:4.11-CF1.1 DATE:2020/12/18 SCORE:-1.74752\n* -1D <体言><用言:判>\n+ 1D <係:文節内><体言><EID:0>\n人工 じんこう 人工 名詞 6 普通名詞 1 * 0 * 0 "代表表記:人工/じんこう カテゴリ:抽象物" \n+ -1D <体言><用言:判><EID:1>\n知能 ちのう 知能 名詞 6 普通名詞 1 * 0 * 0 "代表表記:知能/ちのう カテゴリ:抽象物" \n'
expect = [
    {'surface': '人工', 'base': '人工', 'pos': '名詞', 'pos1': '普通名詞'},
    {'surface': '知能', 'base': '知能', 'pos': '名詞', 'pos1': '普通名詞'},
]
get_morph_dict_list(tes) == expect

True

In [182]:
# 文リストを、文（形態素辞書リスト）リストにする
sentenses_morph_dict = [get_morph_dict_list(s) for s in sentenses]

In [152]:
# 形態素の辞書リストから、インスタンスリストを作る
def get_morph_list(morph_dict_list: List[dict]) -> List[Morph]:
    return [Morph(morph_dict) for morph_dict in morph_dict_list]

tes = [
    {'surface': '人工', 'base': '人工', 'pos': '名詞', 'pos1': '普通名詞'},
    {'surface': '知能', 'base': '知能', 'pos': '名詞', 'pos1': '普通名詞'},
]
get_morph_list(tes)

[<__main__.Morph at 0x1e6467f29b0>, <__main__.Morph at 0x1e6467f2550>]

In [183]:
# 文（形態素辞書リスト）リストを、文（Morphオブジェクトリスト）にする
sentenses_morph = [get_morph_list(s) for s in sentenses_morph_dict]

In [189]:
# 表示
''.join([s.surface for s in sentenses_morph[2]])

'人工知能（じんこうちのう、、AI〈エーアイ〉）とは、「『計算（）』という概念と『コンピュータ（）』という道具を用いて『知能』を研究する計算機科学（）の一分野」を指す語。「言語の理解や推論、問題解決などの知的行動を人間に代わってコンピューターに行わせる技術」、または、「計算機（コンピュータ）による知的な情報処理システムの設計や実現に関する研究分野」ともされる。'

## 41. 係り受け解析結果の読み込み（文節・係り受け）
40に加えて，文節を表すクラスChunkを実装せよ．このクラスは形態素（Morphオブジェクト）のリスト（morphs），係り先文節インデックス番号（dst），係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つこととする．さらに，入力テキストの係り受け解析結果を読み込み，１文をChunkオブジェクトのリストとして表現し，冒頭の説明文の文節の文字列と係り先を表示せよ．本章の残りの問題では，ここで作ったプログラムを活用せよ．